Ada López del Castillo, 1605347

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.datasets import make_blobs, load_breast_cancer
from sklearn.model_selection import (
    train_test_split, StratifiedKFold, cross_val_score, cross_validate, GridSearchCV, LeaveOneOut
)
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import (
    confusion_matrix, accuracy_score, precision_score, recall_score,
    f1_score, matthews_corrcoef, balanced_accuracy_score, roc_auc_score,
    RocCurveDisplay
)
from sklearn.pipeline import Pipeline

# Inferència estadística
from scipy.stats import shapiro, ttest_rel, wilcoxon

### EXERCICI 1: Split vs k-fold CV

Amb el dataset de Breast Cancer, calcula Accuracy, F1 i MCC amb split (80/20) i amb k-fold CV=10 per a SVM(RBF, C=1) i kNN(k=7). Comenta diferències d’estabilitat i possibles biaixos d’estimació.

In [ ]:
data = load_breast_cancer()
X, y = data.data, data.target

#Fem split 80/20
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, stratify=y, random_state=42
)

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test  = scaler.transform(X_test)

#posem KNN=7
svm_base = SVC(kernel='rbf', C=1, probability=True, random_state=42).fit(X_train, y_train)
knn_base = KNeighborsClassifier(n_neighbors=7).fit(X_train, y_train)

yhat_svm = svm_base.predict(X_test)
yhat_knn = knn_base.predict(X_test)

#Volem l'accuracy, f1 i MCC
def metrics_dict(y_true, y_pred):
    return dict(
        accuracy=accuracy_score(y_true, y_pred),
        f1=f1_score(y_true, y_pred),
        mcc=matthews_corrcoef(y_true, y_pred)
    )

print("SPLIT 80/20")
print("Mètriques hold-out — SVM:\n", pd.Series(metrics_dict(y_test, yhat_svm)).round(4))
print("\nMètriques hold-out — kNN:\n", pd.Series(metrics_dict(y_test, yhat_knn)).round(4))

In [ ]:
#Ara amb K-fold CV=10

cv = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)

pipe_svm = Pipeline([
    ("scaler", StandardScaler()),
    ("clf", SVC(kernel='rbf', C=1, random_state=42))
])
pipe_knn = Pipeline([
    ("scaler", StandardScaler()),
    ("clf", KNeighborsClassifier(n_neighbors=7))
])

cv_svm = cross_validate(pipe_svm, X, y, cv=cv, scoring=["accuracy", "f1", "matthews_corrcoef"])
cv_knn = cross_validate(pipe_knn, X, y, cv=cv, scoring=["accuracy", "f1", "matthews_corrcoef"])

print("\nK-fold CV=10:")
print("SVM — Accuracy:", round(cv_svm["test_accuracy"].mean(), 4),
      "f1:", round(cv_svm["test_f1"].mean(), 4),
      "MCC:", round(cv_svm["test_matthews_corrcoef"].mean(), 4))
print("kNN — Accuracy:", round(cv_knn["test_accuracy"].mean(), 4),
      "f1:", round(cv_knn["test_f1"].mean(), 4),
      "MCC:", round(cv_knn["test_matthews_corrcoef"].mean(), 4))


Amb el split 80/20 tenim resultats molt bons en els dos casos, tot i que l'SVM és lleugerament millor que el KNN en totes les mètriques. En el cas de K-fold, el KNN també té valors més baixos que el SVM, i en general són inferiors a les del split.

Recordem que Split Validation és un mètode simple i ràpid, però té poca fiabilitat estadística, ja que els valors depenen de la partició aleatòria escollida, si hagués estat una altra, podrien haver sortit unes mètriques diferents.
En canvi, el K-fold, que consisteix a dividir en 10 folds en aquest cas, és molt més robust, perquè els valors són la mitjana de 10 particions diferents i per tant, es té en compte tota la variabilitat del conjunt de dades.
Per tant, els valors són més estables i representatius.

A més, tant en split com en K-fold, el SVM és més robust i té millor rendiment que el KNN.
Això ho podem veure clarament en el MCC, que és la mètrica que resumeix el rendiment de manera global.

### Exercici 2: Comparació aparellada amb test d'hipòtesis

Compara SVM(RBF, C=1) vs kNN(k=7) amb k=10 CV usant F1. Aplica Shapiro-Wilk a la diferència fold a fold i tria el test adequat (t-test aparellat o Wilcoxon). Interpreta el p-valor.

In [ ]:
def cv_pairwise_compare(X, y, model_a, model_b, k=10, scoring="f1", random_state=42):
    skf = StratifiedKFold(n_splits=k, shuffle=True, random_state=random_state)
    a_scores, b_scores = [], []
    for tr, va in skf.split(X, y):
        Xtr, Xva = X[tr], X[va]
        ytr, yva = y[tr], y[va]
        # Escalat dins de cada fold via Pipeline
        pipe_a = Pipeline([("scaler", StandardScaler()), ("clf", model_a)])
        pipe_b = Pipeline([("scaler", StandardScaler()), ("clf", model_b)])
        pipe_a.fit(Xtr, ytr); pipe_b.fit(Xtr, ytr)
        pa = pipe_a.predict(Xva); pb = pipe_b.predict(Xva)
        if scoring == "accuracy":
            sa = accuracy_score(yva, pa); sb = accuracy_score(yva, pb)
        elif scoring == "precision":
            sa = precision_score(yva, pa); sb = precision_score(yva, pb)
        elif scoring == "recall":
            sa = recall_score(yva, pa); sb = recall_score(yva, pb)
        elif scoring == "balanced_accuracy":
            sa = balanced_accuracy_score(yva, pa); sb = balanced_accuracy_score(yva, pb)
        elif scoring == "mcc":
            sa = matthews_corrcoef(yva, pa); sb = matthews_corrcoef(yva, pb)
        else:  # default F1
            sa = f1_score(yva, pa); sb = f1_score(yva, pb)
        a_scores.append(sa); b_scores.append(sb)
    a_scores, b_scores = np.array(a_scores), np.array(b_scores)
    diff = a_scores - b_scores
    result = {"a_scores": a_scores, "b_scores": b_scores, "diff": diff}
    sw_p = shapiro(diff).pvalue if len(diff) >= 3 else np.nan
    if np.isnan(sw_p) or sw_p < 0.05:
        stat, p = wilcoxon(diff, alternative="greater")
        test = "Wilcoxon (no paramètric) sobre diferències"
    else:
        stat, p = ttest_rel(a_scores, b_scores, alternative="greater")
        test = "t-test aparellat (H1: model A millor)"
    result.update({"test": test, "stat": stat, "pvalue": p, "shapiro_p": sw_p})
    return result

res = cv_pairwise_compare(
    X, y, SVC(kernel='rbf', C=1, random_state=42), KNeighborsClassifier(n_neighbors=7),
    k=10, scoring="f1", random_state=7
)

print("Mitjanes F1  — SVM:", round(res["a_scores"].mean(),4),
      " kNN:", round(res["b_scores"].mean(),4))
print("Diferències (SVM−kNN) per fold:", np.round(res["diff"],4))

print("\nTest:", res["test"])
print("p-valor:", res["pvalue"])
print("Shapiro-Wilk p(normalitat de la diferència):", res["shapiro_p"])
print("Interpretació: p<0.05 → evidència que SVM supera kNN en la mètrica escollida.")

Pel que fa a mitjanes F1, és millor el SVM, però la diferència és molt petita.
En les diferències per fold veiem que són també molt petites, en la majoria de folds la SVM guanya, en la resta, en els que surt 0. hi ha empat i en les que surt en negatiu és que el KNN és superior a SVM.

Aplicant el test estadístic Shapiro-Wilk a les diferències, que serveix per comprovar si el conjunt de dades segueix una distribució normal, ha sortit p: 0.01768 < 0.05, per tant, rebutgem la hipòtesi nul·la sobre que segueixen una distribució normal.

Si hagués sortit un conjunt de dades amb distribució normal, s'hauria d'escollir el t-test aparellat, Wilcoxon és l'alternativa no paramètrica, és per això que el test escollit ha estat el de Wilcoxon.
I ha donat un p-valor de 0.0390625, rebutjant la hipòtesi nul·la, per tant, podem afirmar que SVM supera el kNN en la mètrica escollida, ja que si que hi ha evidència estadistica.

### Exercici 3: Bootstrap d'una mètrica

Estima amb bootstrap (B=300) la distribució de MCC per SVM(RBF, C=1) i kNN(k=7) i calcula IC del 95%. Comenta si els intervals es solapen i què implica.

In [ ]:
def bootstrap_metric(X, y, model, B=300, metric="mcc", random_state=0):
    rng = np.random.default_rng(random_state)
    n = len(X)
    vals = []
    for b in range(B):
        idx = rng.integers(0, n, size=n)
        oob = np.setdiff1d(np.arange(n), np.unique(idx))
        if len(oob) == 0:
            continue
        Xb, yb = X[idx], y[idx]
        Xoob, yoob = X[oob], y[oob]
        pipe = Pipeline([("scaler", StandardScaler()), ("clf", model)])
        pipe.fit(Xb, yb)
        yp = pipe.predict(Xoob)
        if metric == "mcc":
            v = matthews_corrcoef(yoob, yp)
        elif metric == "f1":
            v = f1_score(yoob, yp)
        elif metric == "accuracy":
            v = accuracy_score(yoob, yp)
        else:
            raise ValueError("Mètrica no suportada")
        vals.append(v)
    vals = np.array(vals)
    mean = float(np.mean(vals))
    ci = tuple(np.quantile(vals, [0.025, 0.975]))
    return vals, mean, ci

#Models
m_svm = SVC(kernel='rbf', C=1, random_state=42)
m_knn = KNeighborsClassifier(n_neighbors=7)

#Bootstrap MCC de cada model
svm_vals, svm_mean, svm_ci = bootstrap_metric(X, y, m_svm, B=300, metric="mcc", random_state=1)
knn_vals, knn_mean, knn_ci = bootstrap_metric(X, y, m_knn, B=300, metric="mcc", random_state=1)

print("SVM — MCC: mitjana", round(svm_mean,4), "IC95%", (round(svm_ci[0],4), round(svm_ci[1],4)))
print("kNN — MCC: mitjana", round(knn_mean,4), "IC95%", (round(knn_ci[0],4), round(knn_ci[1],4)))

plt.figure()
plt.hist(svm_vals, bins=20)
plt.title("Bootstrap MCC — SVM")
plt.xlabel("MCC"); plt.ylabel("Freqüència")
plt.show()

plt.figure()
plt.hist(knn_vals, bins=20)
plt.title("Bootstrap MCC — kNN")
plt.xlabel("MCC"); plt.ylabel("Freqüència")
plt.show()

El mètode bootstrap és un model més robust que els anteriors, es crea una mostra aleatòria amb reemplaçament del conjunt original, s'avalua sobre els casos que han quedat fora i es repeteix moltes vegades. D'aquesta manera s'agafa la mitjana de la mètrica, en aquest cas del MCC, i a més ,podem calcular intervals de confiança.

Notem que el SVM té una mitja del MCC més alta que el kNN, cosa que no ens sorprèn veient els exercicis anteriors. I tal com podem veure en els histogrammes, en el cas de SVM els valors es concentren sobre el 0.94 i en el KNN en el 0.92.

Tot i això, mirant els intervals de confiança, veiem que se solapen. Per tant, tot i que la mitjana hagi estat superior en el SVM, no és estadísticament significant.
Ens torna a passar el mateix que abans, amb el bootstrap no podem concloure que SVM sigui millor que KNN en termes de MCC.

### Exercici 4: Dades desequilibrades i mètriques robustes

Simula dades amb 90% de valors en la classe negativa. Avalua kNN(k=3) i SVM(RBF, C=1) amb Accuracy, Recall (classe minoritària), F1, BA i MCC. Explica la paradoxa de l'accuracy i per què MCC/BA són preferibles.

In [ ]:
X, y = make_blobs(
    n_samples=1200,
    centers=2,
    cluster_std=[1.3, 1.3],
    n_features=2,
    random_state=7
)

# Generem un vector de classes amb el desequilibri desitjat
rng = np.random.default_rng(42)
mask = rng.choice([0, 1], size=len(y), p=[0.9, 0.1])  # 90% 0, 10% 1
y = mask

# Split estratificat per mantenir la proporció de classes
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, stratify=y, random_state=42
)

# Models
pipe_svm = Pipeline([
    ("scaler", StandardScaler()),
    ("clf", SVC(kernel="rbf", C=1, random_state=0))
])

pipe_knn = Pipeline([
    ("scaler", StandardScaler()),
    ("clf", KNeighborsClassifier(n_neighbors=3))
])

pipe_svm.fit(X_train, y_train)
pipe_knn.fit(X_train, y_train)

y_predsvm = pipe_svm.predict(X_test)
y_predknn = pipe_knn.predict(X_test)

# 4. Funció per mostrar mètriques
def metrics_dict(y_true, y_pred):
    return {
        "Accuracy": accuracy_score(y_true, y_pred),
        "Recall(minoritària)": recall_score(y_true, y_pred, pos_label=1),
        "F1": f1_score(y_true, y_pred, pos_label=1),
        "Balanced Accuracy": balanced_accuracy_score(y_true, y_pred),
        "MCC": matthews_corrcoef(y_true, y_pred)
    }

print("Mètriques SVM:\n", pd.Series(metrics_dict(y_test, y_predsvm)).round(4))
print("\nMètriques kNN:\n", pd.Series(metrics_dict(y_test, y_predknn)).round(4))

Tenim les 5 mètriques calculades en els dos models.

Si ens fixem en l'accuracy, veiem que ens indica que el SVM és millor, però amb dades desequilibrades, l'accuracy pot indicar que un model va bé tot i no ser el cas. Com en el nostre cas, que tenim una classe majoritària del 90%, el model pot predir sempre la classe negativa i encertar sense fer res.

El recall de la classe minoritària ens confirma que l'accuracy és poc fiable, surt molt baix, indicant que els models a penes detecten la classe minoritària.
El F1 també ens diu que cap dels dos funciona bé, tot i que en el cas de KNN és una mica millor.

Ara, pel que fa al balanced accuracy i el mcc, notem que detecten bé els problemes de desequilibri, en el cas de BA, dona 0.5 en els dos models, indicant que els models no estan classificant res i deixant clar el mal rendiment d'aquests.
I el MCC que és la mètrica que millor capta la situació de manera global, ens diu amb aquests números tan baixos que els models de classificació no van bé, i no estan aprenent res útil.

D'aquesta manera veiem la paradoxa de l'accuracy, si només ens fixem en aquesta mètrica podem pensar que un model funciona bé quan en realitat, és el contrari, per això, és important mirar altres mètriques més completes com poden ser el BA i el MCC.
